### Baixando e extraindo os dados no local do Docker para envio para o HDFS
```sh
cd /input/

curl -O https://mobileapps.saude.gov.br/esus-vepi/files/unAFkcaNDeXajurGB7LChj8SgQYS2ptm/04bd3419b22b9cc5c6efac2c6528100d_HIST_PAINEL_COVIDBR_06jul2021.rar

unrar x 04bd3419b22b9cc5c6efac2c6528100d_HIST_PAINEL_COVIDBR_06jul2021.rar```


## Questão 1

Entrando no container do Namenode, criando a pasta e transferindo os arquivos do local para o HDFS:

```sh
docker exec -it namenode bash

hdfs dfs -mkdir -p /user/projeto_basico/dados 

hdfs dfs -put /input/*.csv /user/projeto_basico/dados
```

In [1]:
# Verificando a transferência 

!hdfs dfs -ls -h /user/projeto_basico/dados

Found 4 items
-rw-r--r--   3 root supergroup     59.6 M 2022-04-25 19:16 /user/projeto_basico/dados/HIST_PAINEL_COVIDBR_2020_Parte1_06jul2021.csv
-rw-r--r--   3 root supergroup     73.0 M 2022-04-25 19:16 /user/projeto_basico/dados/HIST_PAINEL_COVIDBR_2020_Parte2_06jul2021.csv
-rw-r--r--   3 root supergroup     86.9 M 2022-04-25 19:17 /user/projeto_basico/dados/HIST_PAINEL_COVIDBR_2021_Parte1_06jul2021.csv
-rw-r--r--   3 root supergroup      2.9 M 2022-04-25 19:17 /user/projeto_basico/dados/HIST_PAINEL_COVIDBR_2021_Parte2_06jul2021.csv


## Questão 2

In [3]:
from pyspark.sql import SparkSession, Row
from pyspark.sql.functions import *


spark = SparkSession \
    .builder \
    .appName("Projeto Basico") \
    .getOrCreate()

In [4]:
# Lendo os dados em .csv como DataFrame:

df =  spark.read.csv("hdfs:///user/projeto_basico/dados/*.csv", sep=";", header=True, inferSchema=True)

In [4]:
# Criando a database e verificando se existe

spark.sql("CREATE DATABASE IF NOT EXISTS covidbr")
spark.sql("show databases").show()

+------------+
|databaseName|
+------------+
|     covidbr|
|     default|
+------------+



In [ ]:
# Salvando o df no Hive e confirmando que a tabela foi salva:

df_output = df.write.mode("overwrite").partitionBy("municipio").saveAsTable("covidbr.initial_table")

In [11]:
!hdfs dfs -ls -h /user/hive/warehouse/covidbr.db/initial_table/_SUCCESS

-rw-r--r--   2 root supergroup          0 2022-04-23 23:23 /user/hive/warehouse/covidbr.db/initial_table/_SUCCESS


In [12]:
spark.sql("use covidbr")
spark.sql("show tables").show()

+--------+-------------+-----------+
|database|    tableName|isTemporary|
+--------+-------------+-----------+
| covidbr|initial_table|      false|
+--------+-------------+-----------+



In [15]:
df.printSchema()

root
 |-- regiao: string (nullable = true)
 |-- estado: string (nullable = true)
 |-- municipio: string (nullable = true)
 |-- coduf: integer (nullable = true)
 |-- codmun: integer (nullable = true)
 |-- codRegiaoSaude: integer (nullable = true)
 |-- nomeRegiaoSaude: string (nullable = true)
 |-- data: timestamp (nullable = true)
 |-- semanaEpi: integer (nullable = true)
 |-- populacaoTCU2019: integer (nullable = true)
 |-- casosAcumulado: decimal(10,0) (nullable = true)
 |-- casosNovos: integer (nullable = true)
 |-- obitosAcumulado: integer (nullable = true)
 |-- obitosNovos: integer (nullable = true)
 |-- Recuperadosnovos: integer (nullable = true)
 |-- emAcompanhamentoNovos: integer (nullable = true)
 |-- interior/metropolitana: integer (nullable = true)



In [5]:
df.show(5)

+------+------+---------+-----+------+--------------+---------------+-------------------+---------+----------------+--------------+----------+---------------+-----------+----------------+---------------------+----------------------+
|regiao|estado|municipio|coduf|codmun|codRegiaoSaude|nomeRegiaoSaude|               data|semanaEpi|populacaoTCU2019|casosAcumulado|casosNovos|obitosAcumulado|obitosNovos|Recuperadosnovos|emAcompanhamentoNovos|interior/metropolitana|
+------+------+---------+-----+------+--------------+---------------+-------------------+---------+----------------+--------------+----------+---------------+-----------+----------------+---------------------+----------------------+
|Brasil|  null|     null|   76|  null|          null|           null|2020-02-25 00:00:00|        9|       210147125|             0|         0|              0|          0|            null|                 null|                  null|
|Brasil|  null|     null|   76|  null|          null|           null

## Questão 3

In [6]:
# Criando um dataframe para trabalhar as visualizações:

df_visualizacao = df.select("regiao", "data", "municipio", "populacaoTCU2019", "casosAcumulado", "casosNovos",\
                            "obitosAcumulado", "obitosNovos", "Recuperadosnovos", "emAcompanhamentoNovos")

In [7]:
df_visualizacao.show(5)

+------+-------------------+---------+----------------+--------------+----------+---------------+-----------+----------------+---------------------+
|regiao|               data|municipio|populacaoTCU2019|casosAcumulado|casosNovos|obitosAcumulado|obitosNovos|Recuperadosnovos|emAcompanhamentoNovos|
+------+-------------------+---------+----------------+--------------+----------+---------------+-----------+----------------+---------------------+
|Brasil|2020-02-25 00:00:00|     null|       210147125|             0|         0|              0|          0|            null|                 null|
|Brasil|2020-02-26 00:00:00|     null|       210147125|             1|         1|              0|          0|            null|                 null|
|Brasil|2020-02-27 00:00:00|     null|       210147125|             1|         0|              0|          0|            null|                 null|
|Brasil|2020-02-28 00:00:00|     null|       210147125|             1|         0|              0|         

In [6]:
# Criando uma variável para retornar a data mais recente da tabela:

data_max_conv_unix =  df_visualizacao.select(max("data")).\
                       withColumn("unix_time", unix_timestamp(col("max(data)"),"yyyy-MM-dd HH:mm:ss"))

data_max_conv_data = data_max_conv_unix.withColumn("data", from_unixtime("unix_time", "yyyy-MM-dd"))

data_max = data_max_conv_data.select("data").collect()[0][0]

In [17]:
print(data_max)

2021-07-06


In [7]:
#Visualização 1 - Casos Recuperados e em Acompanhamento:

df_vis_1 = df_visualizacao.select(
                       format_number(col("Recuperadosnovos"),0).alias("Casos_Recuperados"),
                       format_number(col("emAcompanhamentoNovos"),0).alias("Em_Acompanhamento")
                       ).filter((col("data") == data_max) & (col("regiao") == "Brasil"))

In [24]:
df_vis_1.show()

+-----------------+-----------------+
|Casos_Recuperados|Em_Acompanhamento|
+-----------------+-----------------+
|       17,262,646|        1,065,477|
+-----------------+-----------------+



In [8]:
# Visualização 2 - CASOS CONFIRMADOS (Acumulados, Novos e Incidência/100.000 Habitantes):

df_vis_2 = df_visualizacao.select(
                        format_number(col("casosAcumulado"), 0).alias("Casos_Acumulados"),
                        format_number(col("casosNovos"),0).alias("Casos_Novos"),
                        format_number((col("casosAcumulado")/col("populacaoTCU2019")*100000),1).alias("Incidência")
                        ).filter((col("data") == data_max) & (col("regiao") == "Brasil"))

In [23]:
df_vis_2.show()

+----------------+-----------+----------+
|Casos_Acumulados|Casos_Novos|Incidência|
+----------------+-----------+----------+
|      18,855,015|     62,504|   8,972.3|
+----------------+-----------+----------+



In [9]:
# Visualização 3 - ÓBITOS CONFIRMADOS (Acumulados, Novos, Letalidade e Mortalidade):

df_vis_3 = df_visualizacao.select(
                        format_number(col("obitosAcumulado"), 0).alias("Óbitos_Acumulados"),
                        format_number(col("obitosNovos"),0).alias("Óbitos_Novos"),
                        format_number((col("obitosAcumulado")/col("casosAcumulado")*100),1).alias("Incidência_[%]"),
                        format_number((col("obitosAcumulado")/col("populacaoTCU2019")*100000),1).alias("Mortalidade")
                        ).filter((col("data") == data_max) & (col("regiao") == "Brasil"))

In [22]:
df_vis_3.show()

+-----------------+------------+--------------+-----------+
|Óbitos_Acumulados|Óbitos_Novos|Incidência_[%]|Mortalidade|
+-----------------+------------+--------------+-----------+
|          526,892|       1,780|           2.8|      250.7|
+-----------------+------------+--------------+-----------+



## Questão 4

In [10]:
# Salvando a primeira vizualização como tabela Hive:

output_tabela_hive = df_vis_1.write.mode("overwrite").saveAsTable("covidbr.visualizacao1")

In [13]:
# Verificando a tabela no Hive:

spark.sql("use covidbr")
spark.sql("show tables").show()
spark.sql("select * from covidbr.visualizacao1").show()

+--------+-------------+-----------+
|database|    tableName|isTemporary|
+--------+-------------+-----------+
| covidbr|initial_table|      false|
| covidbr|visualizacao1|      false|
+--------+-------------+-----------+

+-----------------+-----------------+
|Casos_Recuperados|Em_Acompanhamento|
+-----------------+-----------------+
|       17,262,646|        1,065,477|
+-----------------+-----------------+



## Questão 5

In [ ]:
# Salvando a segunda vizualização em formato parquet e compressão snappy:

output_tabela_casos_confirmados = df_vis_2.write.format("parquet")\
                                .save("/user/projeto_basico/casos_confirmados",compression="snappy")

In [14]:
!hdfs dfs -ls /user/projeto_basico/casos_confirmados

Found 2 items
-rw-r--r--   2 root supergroup          0 2022-04-25 00:30 /user/projeto_basico/casos_confirmados/_SUCCESS
-rw-r--r--   2 root supergroup       1003 2022-04-25 00:30 /user/projeto_basico/casos_confirmados/part-00000-daa4870d-db50-4bea-b78f-a7f9f4ddb1aa-c000.snappy.parquet


## Questão 6

In [15]:
# Salvando a terceira vizualização em um tópico no kafka em formato json:


output_tabela_obitos= df_vis_3.select(to_json(struct("*")).alias("value")).write\
                    .format("kafka")\
                    .option("kafka.bootstrap.servers", "kafka:9092")\
                    .option("topic","obitos-confirmados")\
                    .save()

In [16]:
# Verificando a gravação no tópico kafka:

input_tabela_obitos = spark.read \
                    .format("kafka") \
                    .option("kafka.bootstrap.servers", "kafka:9092") \
                    .option("subscribe", "obitos-confirmados") \
                    .load()

print(input_tabela_obitos.selectExpr("CAST(last(value) AS string)").collect()[0][0])

{"Óbitos_Acumulados":"526,892","Óbitos_Novos":"1,780","Incidência_[%]":"2.8","Mortalidade":"250.7"}


## Questão 7

In [18]:
# Criando a visualização dos dados enviados ao HDFS:

df_vis_4 =  df_visualizacao.select(
                        col('regiao').alias("Regiões"),\
                        col("populacaoTCU2019"),
                        col('municipio').cast("string").alias("cidade"),\
                        col("obitosAcumulado").alias("Óbitos_Acumulados"),\
                        col("casosAcumulado").alias("Casos_Acumulados"),\
                        col("data").alias("Atualização")\
                                    ).dropna(subset=["populacaoTCU2019"]).\
                                    sort(desc("obitosAcumulado")).\
                                    filter((col("data") == data_max) & (col("cidade").isNull()))

In [19]:
df_vis_4.show(5)

+-------+----------------+------+-----------------+----------------+-------------------+
|Regiões|populacaoTCU2019|cidade|Óbitos_Acumulados|Casos_Acumulados|        Atualização|
+-------+----------------+------+-----------------+----------------+-------------------+
| Brasil|       210147125|  null|           526892|        18855015|2021-07-06 00:00:00|
|Sudeste|        45919049|  null|           130389|         3809222|2021-07-06 00:00:00|
|Sudeste|        17264943|  null|            56192|          970268|2021-07-06 00:00:00|
|Sudeste|        21168791|  null|            47148|         1836198|2021-07-06 00:00:00|
|    Sul|        11377239|  null|            31867|         1235914|2021-07-06 00:00:00|
+-------+----------------+------+-----------------+----------------+-------------------+
only showing top 5 rows



In [20]:
# Após filtrar valores repetidos da tabela do HDFS, montado a tabela da visualização 4:

df_final_vis_4 = df_vis_4.groupBy("Regiões").agg(
                        format_number(sum("Casos_Acumulados"),0).alias("Casos_Acumulados"),    
                        format_number(sum("Óbitos_Acumulados"), 0).alias("Óbitos_Acumulados"),
                        format_number((sum("Óbitos_Acumulados")/sum("Casos_Acumulados")*100),1).alias("Incidência_[%]"),
                        format_number((sum("Óbitos_Acumulados")/sum("populacaoTCU2019")*100000),1).alias("Mortalidade"),
                        date_format(max("Atualização"), "dd/MM/yyyy hh:mm").alias("Atualização")                           
                                           ).sort("Regiões")

In [21]:
df_final_vis_4.show()

+------------+----------------+-----------------+--------------+-----------+----------------+
|     Regiões|Casos_Acumulados|Óbitos_Acumulados|Incidência_[%]|Mortalidade|     Atualização|
+------------+----------------+-----------------+--------------+-----------+----------------+
|      Brasil|      18,855,015|          526,892|           2.8|      250.7|06/07/2021 12:00|
|Centro-Oeste|       1,916,619|           49,207|           2.6|      301.9|06/07/2021 12:00|
|    Nordeste|       4,455,737|          107,824|           2.4|      188.9|06/07/2021 12:00|
|       Norte|       1,732,815|           43,845|           2.5|      237.9|06/07/2021 12:00|
|     Sudeste|       7,138,803|          245,311|           3.4|      277.6|06/07/2021 12:00|
|         Sul|       3,611,041|           80,705|           2.2|      269.2|06/07/2021 12:00|
+------------+----------------+-----------------+--------------+-----------+----------------+

